# COMP34812 Natural Language Understanding Courseworklow key lemming an stemming


## Install required packages

In [1]:
!pip install pandas nltk numpy matplotlib scikit-learn sentencepiece tokenizers
!pip install cupy-cuda12x


In [2]:
import pandas as pd
import regex as re
import numpy as np
import nltk
import os
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')
from nltk.corpus import stopwords
from tokenizers import ByteLevelBPETokenizer


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [3]:
if not os.path.exists('glove_embeddings'):
  !wget https://nlp.stanford.edu/data/glove.6B.zip
  !unzip glove.6B.zip -d glove_embeddings

--2025-03-20 17:27:56--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-03-20 17:27:56--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  1.96MB/s    in 6m 41s  

2025-03-20 17:34:38 (2.05 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove_embeddings/glove.6B.50d.txt  
  inflating: glove_embeddings/glove.6

## Load dataset

In [4]:
dev_set = pd.read_csv('dev.csv')
dev_set.head()

,premise,hypothesis,label
0,"By starting at the soft underbelly, the 16,000...","General Nelson A. Miles had 30,000 troops in h...",0
1,"The class had broken into a light sweat, but w...",The class grew more tense as time went on.,1
2,"Samson had his famous haircut here, but he wou...",It was unknown where exactly within the town S...,1
3,A man with a black shirt holds a baby while a ...,A darkly dressed man passes a crying baby to a...,0
4,I know that many of you are interested in addr...,The problems must be addressed,1


In [5]:
train_set = pd.read_csv('train.csv')
train_set.head()

,premise,hypothesis,label
0,yeah i don't know cut California in half or so...,Yeah. I'm not sure how to make that fit. Maybe...,1
1,actual names will not be used,"For the sake of privacy, actual names are not ...",1
2,The film was directed by Randall Wallace.,The film was directed by Randall Wallace and s...,1
3,"""How d'you know he'll sign me on?""Anse studie...",Anse looked at himself in a cracked mirror.,1
4,In the light of the candles his cheeks looked ...,Drew regarded his best friend and noted that i...,1


In [6]:
stop_words = nltk.corpus.stopwords.words('english')
lemmatizer = nltk.stem.WordNetLemmatizer()

def clean_text(text):
    text = str(text)

    text = text.lower()

    text = re.sub(r'[^\w\s]', ' ', text)

    text = nltk.word_tokenize(text)

    processed = []
    for word in text:
        if word in stop_words:
            continue

        word = lemmatizer.lemmatize(word)

        word = word.strip()

        if len(word) < 2:
            continue

        processed.append(word)

    return processed

In [7]:
dev_set['premise_tokens'] = dev_set['premise'].apply(clean_text)
dev_set['hypothesis_tokens'] = dev_set['hypothesis'].apply(clean_text)

train_set['premise_tokens'] = train_set['premise'].apply(clean_text)
train_set['hypothesis_tokens'] = train_set['hypothesis'].apply(clean_text)

In [8]:
dev_set.head()

,premise,hypothesis,label,premise_tokens,hypothesis_tokens
0,"By starting at the soft underbelly, the 16,000...","General Nelson A. Miles had 30,000 troops in h...",0,"[starting, soft, underbelly, 16, 000, troop, g...","[general, nelson, mile, 30, 000, troop, attack]"
1,"The class had broken into a light sweat, but w...",The class grew more tense as time went on.,1,"[class, broken, light, sweat, gasping, air]","[class, grew, tense, time, went]"
2,"Samson had his famous haircut here, but he wou...",It was unknown where exactly within the town S...,1,"[samson, famous, haircut, would, find, hard, r...","[unknown, exactly, within, town, samson, recei..."
3,A man with a black shirt holds a baby while a ...,A darkly dressed man passes a crying baby to a...,0,"[man, black, shirt, hold, baby, blue, shirted,...","[darkly, dressed, man, pass, cry, baby, man, l..."
4,I know that many of you are interested in addr...,The problems must be addressed,1,"[know, many, interested, addressing, issue, le...","[problem, must, addressed]"


In [9]:
train_set.head()

,premise,hypothesis,label,premise_tokens,hypothesis_tokens
0,yeah i don't know cut California in half or so...,Yeah. I'm not sure how to make that fit. Maybe...,1,"[yeah, know, cut, california, half, something]","[yeah, sure, make, fit, maybe, could, cut, cal..."
1,actual names will not be used,"For the sake of privacy, actual names are not ...",1,"[actual, name, used]","[sake, privacy, actual, name, used]"
2,The film was directed by Randall Wallace.,The film was directed by Randall Wallace and s...,1,"[film, directed, randall, wallace]","[film, directed, randall, wallace, star, mel, ..."
3,"""How d'you know he'll sign me on?""Anse studie...",Anse looked at himself in a cracked mirror.,1,"[know, sign, anse, studied, unkempt, clean, re...","[anse, looked, cracked, mirror]"
4,In the light of the candles his cheeks looked ...,Drew regarded his best friend and noted that i...,1,"[light, candle, cheek, looked, even, hollow, t...","[drew, regarded, best, friend, noted, light, l..."


Dataset analysis

In [10]:
# Randomly select 90% of the rows for the training set
train_df = train_set.sample(frac=0.9, random_state=42)
# The rest of the rows will form the test set
test_df = train_set.drop(train_df.index)
train_df.head()



,premise,hypothesis,label,premise_tokens,hypothesis_tokens
2166,"No, he waits until he has had a violent quarre...",He was clear and free from suspicion.,0,"[wait, violent, quarrel, whole, household, cog...","[clear, free, suspicion]"
10333,oh yeah but that's that's a good way to make a...,A marriage can work when you do dishonest thin...,0,"[oh, yeah, good, way, make, big, problem, marr...","[marriage, work, dishonest, thing]"
10835,Land that is acquired for or in connection wit...,Land that is acquired has no connection with i...,0,"[land, acquired, connection, item, general, pp...","[land, acquired, connection, item, general, pp]"
4688,and my wife is from Plains if you know where P...,My wife went to school in Plains.,1,"[wife, plain, know, plain]","[wife, went, school, plain]"
17561,Since 1980 the building has housed the Museum ...,The building has never been able to house any ...,0,"[since, 1980, building, housed, museum, ethnog...","[building, never, able, house, museum]"


# BPE tokinizer (might use)

In [11]:
# Initialize the tokenizer
tokenizer = ByteLevelBPETokenizer()


if not os.path.exists("byte_bpe"):
    os.makedirs("byte_bpe")
    # Combine tokens from premise and hypothesis into one string per sample for both train and dev sets
    train_set['combined_text'] = train_set.apply(
    lambda row: ' '.join(row['premise_tokens'] + row['hypothesis_tokens']), axis=1)
    dev_set['combined_text'] = dev_set.apply(
    lambda row: ' '.join(row['premise_tokens'] + row['hypothesis_tokens']), axis=1)
    # Combine both sets into one list of sentences
    combined_sentences = pd.concat([train_set['combined_text'], dev_set['combined_text']])

    # Save the combined sentences to a file for SentencePiece training
    combined_sentences.to_csv("combined_train_dev.txt", index=False, header=False)
    # Train on your combined file
    tokenizer.train(files="combined_train_dev.txt", vocab_size=10000, min_frequency=2)
    # Save the tokenizer model
    tokenizer.save_model("byte_bpe")

#tokenizer = ByteLevelBPETokenizer("byte_bpe/vocab.json", "byte_bpe/merges.txt")



# TF-IDF word embeddings

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert tokenized text back to strings for TF-IDF processing
train_premise_texts = train_set['premise_tokens'].apply(lambda x: ' '.join(x))
train_hypothesis_texts = train_set['hypothesis_tokens'].apply(lambda x: ' '.join(x))

dev_premise_texts = dev_set['premise_tokens'].apply(lambda x: ' '.join(x))
dev_hypothesis_texts = dev_set['hypothesis_tokens'].apply(lambda x: ' '.join(x))

# Combine premises & hypotheses for TF-IDF training
all_texts = pd.concat([train_premise_texts, train_hypothesis_texts])

# Initialize and fit TF-IDF on training data
tfidf = TfidfVectorizer(max_features=5000)  # Adjust max_features as needed
tfidf.fit(all_texts)

# Transform training and dev sets
train_premise_tfidf = tfidf.transform(train_premise_texts).toarray()
train_hypothesis_tfidf = tfidf.transform(train_hypothesis_texts).toarray()

dev_premise_tfidf = tfidf.transform(dev_premise_texts).toarray()
dev_hypothesis_tfidf = tfidf.transform(dev_hypothesis_texts).toarray()



# Glove embeddings and tfif wieghted word emeddings plus named eneitiy

In [13]:
glove = "./glove_embeddings/glove.6B.100d.txt"
def load_glove(glove_file):
    embeddings_dict = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.strip().split()
            word = values[0]
            vector = np.array(values[1:], dtype=np.float32)  # <-- Convert to float32
            embeddings_dict[word] = vector
    return embeddings_dict

embedding_dim = 100
loaded_glove = load_glove(glove)



In [14]:
def sentence_embedding(tokens, embeddings_dict, embedding_dim):
    valid_embeddings = [embeddings_dict[token] for token in tokens if token in embeddings_dict]
    if not valid_embeddings:
        # Return zero-vector if no embeddings found
        return np.zeros(embedding_dim)
    sentence_emb = np.mean(valid_embeddings, axis=0)
    return sentence_emb

def pairwise_embedding(premise_tokens, hypothesis_tokens, premise_tfidf, hypothesis_tfidf,  embeddings_dict,embedding_dim):
    premise_emb = sentence_embedding(premise_tokens, embeddings_dict,embedding_dim)
    hypothesis_emb = sentence_embedding(hypothesis_tokens, embeddings_dict,embedding_dim)
    # Concatenate multiple useful features
    combined_emb = np.concatenate([
        premise_emb,
        hypothesis_emb,
        np.abs(premise_emb - hypothesis_emb), # capture difference
        premise_emb * hypothesis_emb           # capture interactions
    ]).astype(np.float32)

        # Concatenate TF-IDF features
    combined_emb = np.concatenate([combined_emb, premise_tfidf, hypothesis_tfidf]).astype(np.float32)

    return combined_emb



In [15]:
import numpy as np
from tqdm import tqdm
tqdm.pandas()

# Convert list of numpy arrays into a single 2D numpy array
train_embeddings = np.stack(train_set.apply(
    lambda x: pairwise_embedding(x['premise_tokens'], x['hypothesis_tokens'], train_premise_tfidf[x.name] ,train_hypothesis_tfidf[x.name] , loaded_glove, embedding_dim=100), axis=1))

dev_embeddings = np.stack(dev_set.apply(
    lambda x: pairwise_embedding(x['premise_tokens'], x['hypothesis_tokens'], dev_premise_tfidf[x.name] ,dev_hypothesis_tfidf[x.name], loaded_glove, embedding_dim=100), axis=1))


# Traditional Approach

In [ ]:
import numpy as np
import cupy as cp
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Convert datasets to GPU-supported format (CuPy)
train_embeddings_gpu = cp.asarray(train_embeddings)
dev_embeddings_gpu = cp.asarray(dev_embeddings)

# Convert labels (must be on CPU for sklearn)
train_labels_cpu = np.array(train_set["label"].values)
dev_labels_cpu = np.array(dev_set["label"].values)

# Parameter grid
param_grid = {
    'C': [0.2, 0.3, 0.4],
    'solver': ['lbfgs','saga','sag', 'liblinear' ],
    'max_iter': [500, 600]
}


# GridSearchCV with sklearn
clf = GridSearchCV(LogisticRegression(), param_grid, cv=3, scoring='f1_macro', verbose=1, n_jobs=-1)
clf.fit(train_embeddings_gpu.get(), train_labels_cpu)  # Move CuPy data to CPU before fitting

# Print best parameters
print("Best Parameters:", clf.best_params_)

# Predict using the best estimator
preds = clf.best_estimator_.predict(dev_embeddings_gpu.get())

# Print classification report
print(classification_report(dev_labels_cpu, preds, target_names=['entailment', 'contradiction']))


Fitting 3 folds for each of 24 candidates, totalling 72 fits


# LSTM RNN Apporch


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
import numpy as np

# ✅ Check if GPU is available
device_name = tf.config.list_physical_devices('GPU')
print("Using device:", "GPU" if device_name else "CPU")

# Convert datasets to NumPy arrays (ensure they are float32 for TensorFlow)
train_embeddings_np = np.array(train_embeddings, dtype=np.float32)
dev_embeddings_np = np.array(dev_embeddings, dtype=np.float32)

train_labels_np = np.array(train_set["label"].values, dtype=np.int32)
dev_labels_np = np.array(dev_set["label"].values, dtype=np.int32)



In [ ]:
# ✅ Define the LSTM Model
input_dim = train_embeddings_np.shape[1]  # GloVe embedding size
hidden_dim = 128  # LSTM hidden units
output_dim = 2  # Number of classes (entailment, contradiction)


In [ ]:
model = Sequential([
    LSTM(hidden_dim, return_sequences=False, input_shape=(None, input_dim)),  # LSTM layer
    Dropout(0.2),  # Regularization
    Dense(64, activation='relu'),  # Dense layer
    Dense(output_dim, activation='softmax')  # Output layer (2 classes)
])
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# ✅ Train the model
epochs = 10
batch_size = 32
model.fit(train_embeddings_np, train_labels_np, batch_size=batch_size, epochs=epochs, validation_data=(dev_embeddings_np, dev_labels_np))


In [ ]:
# ✅ Predict on validation set
preds = model.predict(dev_embeddings_np)
preds_class = np.argmax(preds, axis=1)  # Convert probabilities to class labels


In [ ]:
# ✅ Classification Report
print(classification_report(dev_labels_np, preds_class, target_names=['entailment', 'contradiction']))